In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [6]:
chat_history=[]
# chat_history.extend([
#     HumanMessage("Who was the first president of the United States?"),
#     AIMessage("The first president of the United States was George Washington."),
#     HumanMessage("What year was he inaugurated?"),
#     AIMessage("George Washington was inaugurated as president in 1789.")
# ])


In [7]:
chat_history

[]

# Contextualizing the chained question

In [8]:
contextualize_system_prompt = (
    "You must rewrite the user question to be standalone using chat history context. "
    "Return ONLY the rewritten question - nothing else. No explanations, no answers. "
    "Example: If user asks 'Was he the first president?' and history mentions George Washington, "
    "return: 'Was George Washington the first president?'"
)

In [9]:
contextualize_full_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user","{input}")
    ]   
)

In [10]:
from langchain_ollama import ChatOllama
llm=ChatOllama(model="gemma3:1b", temperature=0.5,)

In [11]:
contextualize_chain=contextualize_full_prompt | llm | StrOutputParser()

In [13]:
contextualize_chain.invoke({
    "chat_history": chat_history,
    "input": "Of which country he was president?"
})

'Of which country was he president?'

# Using LLM to answer relevant question

In [14]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2"
)

persist_directory = "vector_store/chroma_vector_store" 
# Load the Chroma vector store from the local directory
vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=huggingface_embeddings,
    collection_name="all_documents"
)
retriever=vector_store.as_retriever(search_kwargs={"k":3},search_type="similarity")


In [15]:
from langchain.chains import create_history_aware_retriever

history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_full_prompt)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Use the following context to answer"),
        ("system","{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user","{input}")
    ]
)

question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [17]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [18]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "What is GreenGrow Innovations?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gemma3:1b")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "What was their first product?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gemma3:1b")
print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: What is GreenGrow Innovations?
AI: Okay, let’s dive into GreenGrow Innovations. Here’s what I’ve gathered based on the context you provided:

GreenGrow Innovations is a company focused on **developing and deploying advanced, sustainable, and intelligent solutions for the smart grid and energy sector.**

Here’s a breakdown of what they generally do and their focus:

*   **Smart Grid Technology:** They specialize in creating technologies to improve the efficiency and reliability of the electrical grid.
*   **Energy Efficiency:** A key part of their work is optimizing energy distribution and reducing waste.
*   **Data Analytics & IoT:** They utilize data from sensors and other devices to create insights and automate processes within the grid.
*   **Innovation:** They are known for their cutting-edge research and development in areas like smart metering, grid monitoring, and predictive maintenance.

**In short, GreenGrow Innovations is a company working to make the electricity grid 

In [19]:
chat_history

[{'role': 'human', 'content': 'What is GreenGrow Innovations?'},
 {'role': 'ai',
  'content': 'Okay, let’s dive into GreenGrow Innovations. Here’s what I’ve gathered based on the context you provided:\n\nGreenGrow Innovations is a company focused on **developing and deploying advanced, sustainable, and intelligent solutions for the smart grid and energy sector.**\n\nHere’s a breakdown of what they generally do and their focus:\n\n*   **Smart Grid Technology:** They specialize in creating technologies to improve the efficiency and reliability of the electrical grid.\n*   **Energy Efficiency:** A key part of their work is optimizing energy distribution and reducing waste.\n*   **Data Analytics & IoT:** They utilize data from sensors and other devices to create insights and automate processes within the grid.\n*   **Innovation:** They are known for their cutting-edge research and development in areas like smart metering, grid monitoring, and predictive maintenance.\n\n**In short, GreenGro